In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [47]:
results = pd.read_csv('mpc_results.csv')
results['timestamp'] = pd.to_datetime(results['timestamp'])

start = pd.Timestamp(year=2018,
                     month=7,
                     day=1,
                     hour=0,
                     minute=0)
end = pd.Timestamp(year=2018,
                     month=8,
                     day=1,
                     hour=0,
                     minute=0)

results = results[(results['timestamp'] > start) & (results['timestamp'] < end)]
results

,Unnamed: 0,battery_bid_fcas_power,battery_bid_fcas_price,battery_bid_fcas_type,battery_bid_energy_power_gen,battery_bid_energy_power_load,battery_bid_energy_price,battery_bid_energy_type,fcas_clearing_price,energy_price,raise_demand,raise_price,soe,timestamp
1432,1432,1.272607e+02,81.00,gen,-3.000000e+02,0.000000e+00,151.21,gen,70.00,151.21,81.00,10.99000,729.000323,2018-07-01 00:30:00
1433,1433,2.000000e+02,81.00,gen,-3.000000e+02,0.000000e+00,153.61,gen,70.00,153.61,81.00,10.99000,429.000647,2018-07-01 01:00:00
1434,1434,2.000000e+02,81.00,gen,-2.900097e+01,0.000000e+00,139.34,gen,70.00,139.34,81.00,9.00000,129.000970,2018-07-01 01:30:00
1435,1435,5.297197e-08,40.67,gen,-1.000003e+02,0.000000e+00,131.75,gen,63.00,131.75,40.67,12.13000,100.000323,2018-07-01 02:00:00
1436,1436,8.424316e-08,40.83,gen,0.000000e+00,6.609794e-07,112.67,load,63.00,112.67,40.83,12.13000,0.000000,2018-07-01 02:30:00
1437,1437,3.766435e-09,41.00,gen,-1.188834e-08,0.000000e+00,93.35,gen,63.00,93.35,41.00,12.13000,0.000000,2018-07-01 03:00:00
1438,1438,8.851534e-09,41.00,gen,-1.192726e-08,0.000000e+00,82.18,gen,63.00,82.18,41.00,14.60000,0.000000,2018-07-01 03:30:00
1439,1439,2.179403e-08,41.00,gen,-2.880467e-08,0.000000e+00,82.12,gen,63.00,82.12,41.00,12.13000,0.000000,2018-07-01 04:00:00
1440,1440,5.036340e-08,41.00,gen,-5.428617e-08,0.000000e+00,78.96,gen,63.00,78.96,41.00,17.50000,0.000000,2018-07-01 04:30:00
1441,1441,1.328561e-07,41.00,gen,0.000000e+00,3.000000e+02,74.59,load,63.00,74.59,41.00,14.90000,0.000000,2018-07-01 05:00:00


In [48]:
def compute_reward(results):
    reward = []
    for i in range(len(results)):
        r = (results['battery_bid_energy_power_gen'].iloc[i] - results['battery_bid_energy_power_load'].iloc[i]) * results['battery_bid_energy_price'].iloc[i]
        if results['battery_bid_fcas_price'].iloc[i] <= results['fcas_clearing_price'].iloc[i]:
            r += results['battery_bid_fcas_price'].iloc[i] * results['battery_bid_fcas_power'].iloc[i]
        reward.append(r)
    results['reward'] = reward
    return results

In [49]:
results = compute_reward(results)
results

,Unnamed: 0,battery_bid_fcas_power,battery_bid_fcas_price,battery_bid_fcas_type,battery_bid_energy_power_gen,battery_bid_energy_power_load,battery_bid_energy_price,battery_bid_energy_type,fcas_clearing_price,energy_price,raise_demand,raise_price,soe,timestamp,reward
1432,1432,1.272607e+02,81.00,gen,-3.000000e+02,0.000000e+00,151.21,gen,70.00,151.21,81.00,10.99000,729.000323,2018-07-01 00:30:00,-4.536300e+04
1433,1433,2.000000e+02,81.00,gen,-3.000000e+02,0.000000e+00,153.61,gen,70.00,153.61,81.00,10.99000,429.000647,2018-07-01 01:00:00,-4.608300e+04
1434,1434,2.000000e+02,81.00,gen,-2.900097e+01,0.000000e+00,139.34,gen,70.00,139.34,81.00,9.00000,129.000970,2018-07-01 01:30:00,-4.040995e+03
1435,1435,5.297197e-08,40.67,gen,-1.000003e+02,0.000000e+00,131.75,gen,63.00,131.75,40.67,12.13000,100.000323,2018-07-01 02:00:00,-1.317504e+04
1436,1436,8.424316e-08,40.83,gen,0.000000e+00,6.609794e-07,112.67,load,63.00,112.67,40.83,12.13000,0.000000,2018-07-01 02:30:00,-7.103290e-05
1437,1437,3.766435e-09,41.00,gen,-1.188834e-08,0.000000e+00,93.35,gen,63.00,93.35,41.00,12.13000,0.000000,2018-07-01 03:00:00,-9.553525e-07
1438,1438,8.851534e-09,41.00,gen,-1.192726e-08,0.000000e+00,82.18,gen,63.00,82.18,41.00,14.60000,0.000000,2018-07-01 03:30:00,-6.172693e-07
1439,1439,2.179403e-08,41.00,gen,-2.880467e-08,0.000000e+00,82.12,gen,63.00,82.12,41.00,12.13000,0.000000,2018-07-01 04:00:00,-1.471885e-06
1440,1440,5.036340e-08,41.00,gen,-5.428617e-08,0.000000e+00,78.96,gen,63.00,78.96,41.00,17.50000,0.000000,2018-07-01 04:30:00,-2.221536e-06
1441,1441,1.328561e-07,41.00,gen,0.000000e+00,3.000000e+02,74.59,load,63.00,74.59,41.00,14.90000,0.000000,2018-07-01 05:00:00,-2.237700e+04


In [50]:
cumulative_reward = results['reward'].rolling(window=48).mean()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results['timestamp'], y=cumulative_reward, name='reward'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="Cumulative Reward",
    # xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Cumulative Reward")
fig.show()
# fig.write_image('cum_reward_mpc.png')

In [51]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'], y=results['soe'], name='soe'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'], y=results['energy_price'], name='energy price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [52]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_fcas_power'], name='raise power'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['raise_price'], name='raise price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [53]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_energy_power_load'], name='energy power load'), secondary_y=False)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_energy_power_gen'], name='energy power gen'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['energy_price'], name='energy price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Energy Power (MW)", secondary_y=False)
fig.update_yaxes(title_text="Energy Price ($/MW)", secondary_y=True)

fig.show()

In [39]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['battery_bid_fcas_price'], name='fcas bid'), secondary_y=True)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_fcas_power'], name='fcas power'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['fcas_clearing_price'], name='fcas cleared'), secondary_y=True)
#fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['raise_demand'], name='fcas demand'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Fcas Price ($/MW)", secondary_y=True)
fig.update_yaxes(title_text="Fcas Demand (MW)", secondary_y=False)

fig.show()